In [1]:

!pip install torch-geometric torch-scatter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 13.4 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=7f07beece81c2093b7aa3c71d13bc6f2558e3946c9fcbca370b7c2698d0ced8a
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
  Created wheel for torch-scatter: filename=torch_scatter-2.1.1-cp310-cp310-linux_x86_64.whl size=3751354 sha256=bca404288b5cd7be84d2b771b06d9c6f694f3d17dbcae558d5626d3ed9e789d7
  Stored in directory: /root/.cache/pip/wheels/ef/67/58/6566a3b61c6ec0f2ca0c2c324cd035ef2955601f0fb3197d5f
Successfully built torch-geometric torch-scatter


In [2]:

import numpy as np  # 🧮 NumPy for numerical computations
import pandas as pd  # 🐼 Pandas for data manipulation
import os  # 📂 Operating system-related functions
from tqdm import tqdm  # 🔄 tqdm for progress bar visualization

import sklearn  # 🧬 scikit-learn for machine learning utilities
import sklearn.model_selection  # 📊 scikit-learn's model selection module
import torch  # 🔥 PyTorch for deep learning
from torch import nn  # 🧠 PyTorch's neural network module
from torch import Tensor  # 🚀 PyTorch's Tensor data type
from torch_geometric.nn import GCNConv  # 📊 Graph Convolutional Network layer
from torch_geometric.datasets import Planetoid  # 🌍 PyTorch Geometric dataset for graph data
from torch.utils.data import DataLoader, Dataset  # 📦 PyTorch data loading utilities
from timm.scheduler import CosineLRScheduler  # 📈 Learning rate scheduler
import matplotlib.pyplot as plt  # 📊 Matplotlib for plotting

device = 'cuda' if torch.cuda.is_available() else 'cpu'  # ⚙️ Determine if CUDA (GPU) is available


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# 📁 Define a Function to Load DataFrames
# This function loads data stored in different splits (train, valid, test) from a specified directory.
# It reads files in the directory, extracts data using NumPy, and organizes it into DataFrames.

def load_df(directory):
    splits = ["train", "valid", "test"]  # 🔄 List of data splits
    dfs = dict()  # 📊 Dictionary to store DataFrames for each split
    
    for split in splits:
        path = os.path.join(directory, split)  # 📂 Define the path to the split's directory
        files = os.listdir(path)  # 🗂 Get a list of files in the split's directory
        list_df = []  # 📄 List to store data dictionaries
        
        for file in files:
            d = dict(np.load(os.path.join(path, file)))  # 📦 Load data using NumPy
            d['file'] = file  # 📄 Include the file name in the data dictionary
            list_df.append(d)  # 🧾 Append the data dictionary to the list
        dfs[split] = pd.DataFrame.from_dict(list_df)  # 🐼 Create a DataFrame from the list of data dictionaries and store it in the dictionary
    return dfs

# 📄 Load data using the defined function and store it in the 'tile_xla' variable
tile_xla = load_df("/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/")


# 📦 Define Dataset and Model

In [4]:
# 📦 Define Custom Dataset Class
# This class, 'TileDataset', is a custom dataset class for our machine learning task.
# It inherits from the PyTorch 'Dataset' class and implements the necessary methods (__init__, __len__, and __getitem__).

class TileDataset(Dataset):
    def __init__(self, df):
        self.df = df  # 💼 Initialize the dataset with a DataFrame containing the data

    def __len__(self):
        return len(self.df)  # 🔢 Define the length of the dataset, which is the number of rows in the DataFrame

    def __getitem__(self, idx):
        row = self.df.iloc[idx]  # 📄 Get a specific row from the DataFrame based on the provided index
        config_feat = torch.tensor(row['config_feat'].astype(np.float32))  # 🧮 Convert and store 'config_feat' as a PyTorch tensor
        node_feat = torch.tensor(row['node_feat'].astype(np.float32))  # 🧮 Convert and store 'node_feat' as a PyTorch tensor
        node_opcode = torch.tensor(row['node_opcode'].astype(np.int32))  # 🧮 Convert and store 'node_opcode' as a PyTorch tensor
        edge_index = torch.tensor(np.swapaxes(row['edge_index'],0,1).astype(np.int32))  # 🧮 Convert and store 'edge_index' as a PyTorch tensor with axis swapping
        target = (row['config_runtime'] / (row['config_runtime_normalizers'] + 1e-5)).astype(np.float32)  # 📈 Calculate and store the target value with preprocessing
        # 📊 Min-max scale the target value to ensure it's within a specific range (standardization)
        target = (target - np.mean(target)) / (np.std(target) + 1e-5)
        target = torch.tensor(target)  # 🧮 Convert and store the target as a PyTorch tensor
        return config_feat, node_feat, node_opcode, edge_index, target  # 🔁 Return the data and target for a specific sample

# This class defines the structure of our custom dataset, converting and preprocessing data as necessary for training and evaluation.
# The relevant emojis provide a visual context for each part of the code.


In [5]:
# 🧠 Define Simple Neural Network Model
# In this cell, we define a simple neural network model named 'SimpleModel'.
# This model takes input data with specified dimensions and passes it through convolutional and dense layers.

class SimpleModel(torch.nn.Module):
    def __init__(self, hidden_channels, graph_feats, hidden_dim):
        super().__init__()  # 🧬 Initialize the parent class 'torch.nn.Module'
        
        op_embedding_dim = 4  # I choose 4-dimensional embedding
        self.embedding = torch.nn.Embedding(120,  # 120 different op-codes
                                            op_embedding_dim,
                                           )
        assert len(hidden_channels) > 0
        in_channels = op_embedding_dim + 140
        self.convs = torch.nn.ModuleList()
        last_dim = hidden_channels[0]
        
        # Create a sequence of Graph Convolutional Network (GCN) layers
        self.convs.append(GCNConv(in_channels, hidden_channels[0]))
        for i in range(len(hidden_channels) - 1):
            self.convs.append(GCNConv(hidden_channels[i], hidden_channels[i+1]))
            last_dim = hidden_channels[i+1]
        self.convs.append(GCNConv(last_dim, graph_feats))
        
        # Define a sequential dense neural network
        self.dense = torch.nn.Sequential(nn.Linear(graph_feats + 24, 64),
                                         nn.ReLU(),
                                         nn.Linear(64, 64),
                                         nn.ReLU(),
                                         nn.Linear(64, 1),
                                        )

    def forward(self, x_cfg: Tensor, x_feat: Tensor, x_op: Tensor, edge_index: Tensor) -> Tensor:
        
        # Get graph features
        x = torch.cat([x_feat, self.embedding(x_op)], dim=1)  # 📊 Concatenate input features with opcode embeddings
        
        # Pass data through convolutional layers
        for conv in self.convs:
            x = conv(x, edge_index).relu()
        
        # Get 1D graph embedding using average pooling
        x_graph = torch.mean(x, 0)
        
        # Combine graph data with config data
        x = torch.cat([x_cfg, x_graph.repeat((len(x_cfg), 1))], axis=1)  # 🔄 Concatenate config data with repeated graph embeddings
        
        # Pass the combined data through the dense neural network
        x = torch.flatten(self.dense(x))
        
        # Standardize the output
        x = (x - torch.mean(x)) / (torch.std(x) + 1e-5)
        return x

# Create an instance of the 'SimpleModel' and move it to the specified device (CPU or GPU)
model = SimpleModel(hidden_channels=[16, 32, 16, 48], graph_feats=64, hidden_dim=64).to(device)


# 🚂 Train  Epoch

In [6]:
# 📊 Concatenate DataFrames
# In this cell, we concatenate DataFrames 'train' and 'valid' from the 'tile_xla' dictionary along the row axis.
# We then reset the index of the resulting DataFrame for consistent indexing.

# Concatenate 'train' and 'valid' DataFrames along the row axis and reset the index
df = pd.concat((tile_xla["train"], tile_xla["valid"]), axis=0).reset_index(drop=True)

# This operation combines the training and validation data for further processing, ensuring a unified DataFrame.


In [7]:
# 🔄 Cross-Validation Training Loop (Enhanced)

# Define the score_tile_mean function
def score_tile_mean(predictions, df):
    score = 0
    for i in range(len(df)):
        predbest = np.mean(df.iloc[i]['config_runtime'][predictions[i]])
        best = np.mean(np.sort(df.iloc[i]['config_runtime'])[:5])
        score += 2 - predbest / best
    score /= len(df)
    return score

# Define the score_tile_max function
def score_tile_max(predictions, df):
    score = 0
    for i in range(len(df)):
        predbest = np.min(df.iloc[i]['config_runtime'][predictions[i]])
        best = np.min(df.iloc[i]['config_runtime'])
        score += 2 - predbest / best
    score /= len(df)
    return score

# Create a K-Fold cross-validator with 5 splits
kfold = sklearn.model_selection.KFold(n_splits=5, shuffle=True, random_state=0)

# Lists to store mean and max scores for each fold
score_means = []
score_maxs = []

# Define hyperparameters
learning_rate = 5e-4  # Adjust the learning rate to a different value
weight_decay = 1e-6  # Adjust weight decay to a different value
num_epochs = 90  # You can keep the number of epochs as 90 or adjust as needed


# Iterate through each fold
for fold, (tr_idx, va_idx) in enumerate(kfold.split(df)):
    train_dataset = TileDataset(df.iloc[tr_idx])
    val_dataset = TileDataset(df.iloc[va_idx])
    criterion = torch.nn.MSELoss()
    steps = len(train_dataset) * num_epochs  # Update the number of training steps
    warmup_steps = int(steps * 0.1)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = CosineLRScheduler(optimizer, t_initial=steps, warmup_t=warmup_steps, warmup_lr_init=1e-6, lr_min=2e-8)

    best_score = 0
    best_score_max = 0

    # Training loop with increased epochs
    for epoch in range(num_epochs):
        model.train()
        pbar = tqdm(range(len(train_dataset)), leave=False)
        loss_sum = 0
        n = 0
        
        for i in pbar:
            cfg_ft, nd_ft, nd_op, ind, target = train_dataset[i]
            cfg_ft, nd_ft, nd_op, ind, target = cfg_ft.to(device), nd_ft.to(device), nd_op.to(device), ind.to(device), target.to(device)
            
            out = model(cfg_ft, nd_ft, nd_op, ind)
            loss = criterion(out, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1e-2)
            scheduler.step(i + len(train_dataset) * epoch)
            optimizer.step()
            loss_sum += loss.item()
            n += 1
            pbar.set_description(f'running loss: {(loss_sum/n):.2f}, current loss: {(loss.item()):.2f}')
        pbar.close()
        model.eval()
        tile_xla_predictions = []
        pbar = tqdm(range(len(val_dataset)), leave=False)
        
        for i in pbar:
            cfg_ft, nd_ft, nd_op, ind, target = val_dataset[i]
            cfg_ft, nd_ft, nd_op, ind, target = cfg_ft.to(device), nd_ft.to(device), nd_op.to(device), ind.to(device), target.to(device)
            
            out = model(cfg_ft, nd_ft, nd_op, ind)
            tile_xla_predictions.append(np.argsort(out.cpu().detach().numpy())[:5])
        pbar.close()
        
        # Calculate and display scores for the current fold and epoch
        score_mean = score_tile_mean(tile_xla_predictions, val_dataset.df)
        score_max = score_tile_max(tile_xla_predictions, val_dataset.df)
        print(f'fold {fold} epoch {epoch}, comp_score = {score_max:.3f}, mean_score = {score_mean:.3f},')
        
        # Update best scores and save the model if the mean score improves
        if score_mean > best_score:
            best_score = score_mean
            best_score_max = score_max
            torch.save(model.state_dict(), f'best_model_{fold}.pth')
    
    # Append the best scores for this fold to the respective lists
    score_means.append(best_score)
    score_maxs.append(best_score_max)

# Calculate and display the mean scores across all folds
print(f'comp_score = {np.mean(score_maxs)}, mean_score = {np.mean(score_means)},')


fold 0 epoch 0, comp_score = 0.611, mean_score = 0.020,


fold 0 epoch 1, comp_score = 0.602, mean_score = 0.028,


fold 0 epoch 2, comp_score = 0.757, mean_score = 0.205,


fold 0 epoch 3, comp_score = 0.649, mean_score = 0.113,


fold 0 epoch 4, comp_score = 0.837, mean_score = 0.648,


...

fold 4 epoch 85, comp_score = 0.912, mean_score = 0.812,


fold 4 epoch 86, comp_score = 0.912, mean_score = 0.810,


fold 4 epoch 87, comp_score = 0.911, mean_score = 0.809,


fold 4 epoch 88, comp_score = 0.912, mean_score = 0.808,


fold 4 epoch 89, comp_score = 0.911, mean_score = 0.807,
comp_score = 0.9217602880169198, mean_score = 0.8337294808782281,


# 📊 Evaluate on Validation Dataset

# 🚀 Predict and Submit (only tile:xla predictions)

In [8]:
# 📊 Predict on Test Dataset (tile:xla)
# In this section, we use the trained model to make predictions on the test dataset ('tile:xla').

# Create a TileDataset for the 'tile:xla' test dataset
dataset = TileDataset(tile_xla["test"])

# List to store model predictions for each sample in the test dataset
tile_xla_predictions = [[] for i in range(len(dataset))]

# Iterate through each fold (previously trained models)
for fold in range(5):
    # Load the trained model weights for the current fold
    model.load_state_dict(torch.load(f'/kaggle/working/best_model_{fold}.pth'))
    model.eval()  # 🕵️ Set the model to evaluation mode
    pbar = tqdm(range(len(dataset)))  # Progress bar for test data prediction
    
    for i in pbar:
        cfg_ft, nd_ft, nd_op, ind, target = dataset[i]
        cfg_ft, nd_ft, nd_op, ind, target = cfg_ft.to(device), nd_ft.to(device), nd_op.to(device), ind.to(device), target.to(device)

        out = model(cfg_ft, nd_ft, nd_op, ind)
        tile_xla_predictions[i].append(out.cpu().detach().numpy())

# Aggregate predictions by taking the mean and selecting the top 5
tile_xla_predictions = [np.argsort(np.mean(pred, axis=0))[:5] for pred in tile_xla_predictions]

# The 'tile_xla_predictions' now contains the top 5 predicted results for each sample in the 'tile:xla' test dataset.
tile_xla_predictions


100%|██████████| 844/844 [00:03<00:00, 226.66it/s]


[array([746, 554, 127, 408, 731]),
 array([ 415, 5420, 7006, 6480, 5213]),
 array([1344,  161,  206, 1409,  935]),
 array([158, 210, 212,  79, 120]),
 array([222, 164, 124,  31, 139]),
 array([ 611,  565, 1111,  719,  912]),
 array([184, 216, 146, 158, 221]),
 array([6768, 3649,  219,  323, 4885]),
 array([ 12,  49,  16, 115,  27]),
 array([ 74, 123,   2, 115,  61]),
 array([ 84, 519, 691, 121, 113]),
 array([ 94, 196, 241, 403, 363]),
 array([111,  30,  28,  78, 107]),
 array([1558, 7899, 1335, 8265, 7565]),
 array([59, 62, 63, 92, 99]),
 array([124, 132, 129,   7,   8]),
 array([  24,  262, 1495,  979,  521]),
 array([5576, 3856, 5896,  302, 2465]),
 array([229, 180, 437, 384, 339]),
 array([167,  17, 430, 360, 143]),
 array([69, 13,  0,  2, 73]),
 array([ 26, 222, 491, 383, 304]),
 array([5674, 6096,  486, 6254,  886]),
 array([20, 22,  9, 10,  2]),
 array([2199, 8680, 4033, 5925, 1843]),
 array([295, 327, 509, 677, 329]),
 array([  2, 208, 138, 227,  77]),
 array([2513, 1970,  675,

In [9]:
# 📊 Generate and Save Submission File
# In this section, we generate a submission file based on the model predictions and save it.

# Read the sample submission file
sub = pd.read_csv('/kaggle/input/predict-ai-model-runtime/sample_submission.csv')

# Iterate through the test file names and update the submission file with top predictions
for i, filename in enumerate(tile_xla["test"]['file'].values):
    id = 'tile:xla:' + filename[:-4]  # Construct the ID for the submission
    sub.loc[sub.ID == id, 'TopConfigs'] = ';'.join(tile_xla_predictions[i].astype(str))

# Save the updated submission file as 'submission.csv' without the index
sub = sub[sub.ID.str.contains('tile')].reset_index(drop=True)
sub.to_csv('inference_tile_xla.csv', index=False)

# Display the updated submission file
sub


,ID,TopConfigs
0,tile:xla:d6f5f54247bd1e58a10b9e7062c636ab,0;22;21;20;19
1,tile:xla:e3a655daa38e34ec240df959b650ac16,124;802;1301;711;385
2,tile:xla:f8c2c1a1098b2a361c26df668b286c87,27;129;12;202;2
3,tile:xla:4dd1716853ed46ee4e7d09ede1732de8,903;1015;3939;7320;4945
4,tile:xla:d0a69155b6340748c36724e4bfc34be3,641;264;252;640;248
...,...,...
889,layout:nlp:random:60880ed76de53f4d7a1b960b24f2...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
890,layout:nlp:random:23559853d9702baaaacbb0c83fd3...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
891,layout:nlp:random:f6c146fc5cf10be4f3accbaca989...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
892,layout:nlp:random:32531d07a084b319dce484f53a4c...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
